# Imports

In [52]:
print('Loading libraries')
suppressPackageStartupMessages({
    library(data.table)
    library(ggplot2)
    library(ggpubr)
    library(dplyr)
    library(stringr)
    library(rstatix)
    library(tidyr)
    }
)
print('Library load finished')

hpc =  TRUE
work_desktop = FALSE
work_linux = TRUE
singularity = TRUE
prj =  "HCC-CBS-173-Hillman-BFerris-NRG-HN003-Vectra"
if (!hpc){
    if (work_desktop){
        if (work_linux){
            base = paste0('/mnt/d/OneDrive - University of Pittsburgh/Internal_Project_Data_2/' ,prj)
        }else{
            base =paste0('D:\\OneDrive - University of Pittsburgh\\Internal_Project_Data_2\\', prj)
        }
    }else{
        base = paste0('/home/brian/data/odrive/Internal_Project_Data_2/',prj)
    }
}else{
    if (singularity){
        base = '/mnt'
    }else{
        base = paste0('/ix/rbao/Projects/', prj)
    }
}
print(base)
inform.data <- file.path(base,'data','inform_data_exports')
results <- file.path(base,'results')
sampleinfo <- file.path(base,'sampleinfo')
print(results)


[1] "Loading libraries"
[1] "Library load finished"
[1] "/mnt"
[1] "/mnt/results"


## Load data

In [134]:
fn <- 'NRG-HN003-7552156cell-measurements_11col.csv'
start <- Sys.time()
df <- read.csv(file.path(results,fn))
df$pdl1.cell <- 'other'
df$pdl1.cell[df$is.pdl1.cell] = 'all.pdl1'

df$panck.pdl1.cell <-'other'
df$panck.pdl1.cell[df$is.panck.cell & df$is.pdl1.cell] = 'panck.pdl1'
idx <- !((df$combined.pheno %like% 'CK+') | (df$combined.pheno %like% 'CD3+'))
df$panck.pdl1.cell[idx & df$is.pdl1.cell] <- 'panckneg.cd3neg.pdl1'

stop <- Sys.time()
print(paste((stop-start),'seconds'))
head(df)

[1] "48.0321016311646 seconds"


,X,Sample.Name,fn,Cell.ID,cell.type,all.tumor.stroma,tissue.compartment,pdl1.expression,pd1.expression,combined.pheno,is.pdl1.cell,is.panck.cell,pdl1.cell,panck.pdl1.cell
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<chr>,<chr>
1,1,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39799,other,Tumor,inner.tumor.inv.margin,0.125,0.316,other,FALSE,FALSE,other,other
2,2,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39800,panck,Tumor,central.tumor,0.360,0.107,CK+,FALSE,TRUE,other,other
3,3,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39801,other,Tumor,central.tumor,0.220,0.172,other,FALSE,FALSE,other,other
4,4,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39802,panck,Tumor,central.tumor,0.397,0.157,CK+,FALSE,TRUE,other,other
5,5,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39803,other,Tumor,central.tumor,0.111,0.235,PD1+,FALSE,FALSE,other,other
6,6,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39804,other,Tumor,central.tumor,0.420,0.154,other,FALSE,FALSE,other,other


In [195]:
df$pdl1.cell <- 'other'
df$pdl1.cell[df$is.pdl1.cell] = 'all.pdl1'

df$panck.pdl1.cell <-'other'
df$panck.pdl1.cell[df$is.panck.cell & df$is.pdl1.cell] = 'panck.pdl1'
idx <- !((df$combined.pheno %like% 'CK+') | (df$combined.pheno %like% 'CD3+'))
df$panck.pdl1.cell[idx & df$is.pdl1.cell] <- 'panckneg.cd3neg.pdl1'

df %>% count(panck.pdl1.cell)

panck.pdl1.cell,n
<chr>,<int>
other,5478651
panck.pdl1,1123403
panckneg.cd3neg.pdl1,950102


# Subset data and debug necessary table operations

In [136]:
idx <- df$Sample.Name %in% c('RACTIH_Scan1.qptiff','RACTKE_Scan1.qptiff')
sub <- df[idx,]
dim(sub)

[1] 1154004      14

In [6]:
colnames(sub)

[1] "X"                  "Sample.Name"        "fn"                
 [4] "Cell.ID"            "cell.type"          "all.tumor.stroma"  
 [7] "tissue.compartment" "pdl1.expression"    "pd1.expression"    
[10] "combined.pheno"     "is.pdl1.cell"       "is.panck.cell"

In [60]:
sub %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) 

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.


Sample.Name,tissue.compartment,cell.type,n,percent_density
<chr>,<chr>,<chr>,<int>,<dbl>
RACTIH_Scan1.qptiff,central.tumor,cd8.t,4,0.0354
RACTIH_Scan1.qptiff,central.tumor,other,7191,63.5640
RACTIH_Scan1.qptiff,central.tumor,panck,4118,36.4006
RACTIH_Scan1.qptiff,central.tumor,treg,0,0.0000
RACTIH_Scan1.qptiff,distal.stroma,cd8.t,0,0.0000
RACTIH_Scan1.qptiff,distal.stroma,other,3062,91.5396
RACTIH_Scan1.qptiff,distal.stroma,panck,283,8.4604
RACTIH_Scan1.qptiff,distal.stroma,treg,0,0.0000
RACTIH_Scan1.qptiff,inner.tumor.inv.margin,cd8.t,3,0.0133


# Example for pdl1 expression

In [125]:
sub %>% 
    group_by(Sample.Name, tissue.compartment) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             fill = list(median_pdl1_expression = 0)) %>%    
    pivot_wider(names_from = c("tissue.compartment"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())
    # group_by(Sample.Name, tissue.compartment) %>%
    # mutate(percent_density = round(n / sum(n) * 100, digits=4)) 

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,inner.tumor.inv.margin_pdl1.expression,outer.tumor.inv.margin_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.557,0.6635
RACTKE_Scan1.qptiff,4.744,4.0880


In [64]:
sub %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other"))
    

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.


Sample.Name,all.stroma_cd8.t,all.stroma_panck,all.stroma_treg,all.tumor_cd8.t,all.tumor_panck,all.tumor_treg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.0092,9.8999,0.0046,0.0206,44.0977,0.0000
RACTKE_Scan1.qptiff,1.5515,24.6287,3.0400,0.0413,80.8149,0.0724


# Full table generation all.measures

In [228]:
###################### # Calculate values for all tumor / stroma
#TREG CD8T
t.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1
panck.pdl1.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, panck.pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name")

# ALL PDL1
all.pdl1.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")


###################### Calculate density in inner / outer tumor invasive margin
#TREG CD8T
t.dense.in.out <- df %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#ALL PDL1+
all.pdl1.dense.in.out <- df %>% group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1+
panck.pdl1.dense.in.out <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#################### # Calculate density for all tumor invasive margin
#TREG CD8T
t.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

# ALL PDL1
all.pdl1.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4))  %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1+
panck.pdl1.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, panck.pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4))  %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name")

#################### # Calculate density for all tissue, all PDL1
#Treg CD8T
t.dense.all.tissue <- df %>%  group_by(Sample.Name, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>%
    rename_with(~paste0('all.tissue_',.),everything())


#ALL PDL1+
all.pdl1.dense.all.tissue <- df %>% group_by(Sample.Name, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") %>%
    rename_with(~paste0('all.tissue_',.),everything())

#PANCK+/- PDL1+
panck.pdl1.dense.all.tissue <- df %>%  group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  
    tibble::column_to_rownames(var = "Sample.Name") %>%
    rename_with(~paste0('all.tissue_',.),everything())

all.dense <- bind_cols(t.dense.all.tissue, all.pdl1.dense.all.tissue, panck.pdl1.dense.all.tissue, 
                   t.dense.all.ts, all.pdl1.dense.all.ts, panck.pdl1.dense.all.ts, 
                   t.dense.in.out, all.pdl1.dense.in.out, panck.pdl1.dense.in.out,
                   t.dense.all.tiv, all.pdl1.dense.all.tiv, panck.pdl1.dense.all.tiv) %>%
        rename_with(~paste0(.,'.cell_percent.density'),everything())
# head(all.dense)


#################################################################
#####################################################################
###################### PDL1 expression in tumor compartments

all.pdl1.exp.in.out <- df   %>%
    group_by(Sample.Name, tissue.compartment) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             fill = list(median_pdl1_expression = 0))  %>%    
    pivot_wider(names_from = c("tissue.compartment"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all'),everything())

panck.pdl1.exp.in.out <- df   %>%
    group_by(Sample.Name, tissue.compartment,panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%    
    pivot_wider(names_from = c("tissue.compartment","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name") #%>% 
    # rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.tiv <- df   %>% 
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin), 
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all'),everything())

panck.pdl1.exp.all.tiv <- df   %>% 
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0)) %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") #%>% 
    # rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.ts <- df   %>% 
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.tumor.stroma  = unique(tissue.all.tumor.stroma ), 
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.tumor.stroma"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all'),everything())

panck.pdl1.exp.all.ts <- df   %>% 
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, panck.pdl1.cell ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.tumor.stroma  = unique(tissue.all.tumor.stroma ),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")# %>% 
    # rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.tissue <- df  %>%  
    group_by(Sample.Name) %>% 
    summarize(all.tissue=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             fill = list(all.tissue = 0))  %>%   
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all'),everything())

panck.pdl1.exp.all.tissue <- df  %>%  
    group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(all.tissue=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(all.tissue = 0))  %>%   
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "all.tissue") %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    select(-matches("other")) %>%
    rename_with(~paste0('all.tissue_',.),everything())

pdl1.expression <- bind_cols(all.pdl1.exp.all.tissue, panck.pdl1.exp.all.tissue,
                             all.pdl1.exp.all.ts, panck.pdl1.exp.all.ts,
                             all.pdl1.exp.all.tiv, panck.pdl1.exp.all.tiv,
                             all.pdl1.exp.in.out, panck.pdl1.exp.in.out                             
                             ) %>% 
                    rename_with(~paste0(.,'.cell_pdl1.expression'),everything())

all.measures <- bind_cols(all.dense, pdl1.expression)
t<-c()
for (nm in row.names(all.measures)){
    t <- c(t,str_split_fixed(nm,'_',n=2)[1])
}
row.names(all.measures) <- t
all.measures <- all.measures[,sort(colnames(all.measures))]
head(all.measures)

fn <- file.path(results,sprintf('NRG-HN003.sample.values.%d.%d.csv',
                                dim(all.measures)[1],
                                dim(all.measures)[2]))
print(fn)
write.csv(all.measures,fn)
print('Finished')
                   

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped

,all.stroma_all.cell_pdl1.expression,all.stroma_all.pdl1.cell_percent.density,all.stroma_cd8.t.cell_percent.density,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panck.pdl1.cell_percent.density,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_percent.density,all.stroma_treg.cell_percent.density,all.tissue_all.cell_pdl1.expression,all.tissue_all.pdl1.cell_percent.density,⋯,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,inner.tumor.inv.margin_treg.cell_percent.density,outer.tumor.inv.margin_all.cell_pdl1.expression,outer.tumor.inv.margin_all.pdl1.cell_percent.density,outer.tumor.inv.margin_cd8.t.cell_percent.density,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_percent.density,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,outer.tumor.inv.margin_treg.cell_percent.density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.592,2.4819,0.0092,3.0460,0.3137,4.3280,2.1497,0.0046,0.535,1.9324,⋯,0.6375,0.0000,0.6635,2.6947,0.0109,3.166,0.3655,4.2080,2.3074,0.0055
RACTKE,2.937,36.9659,1.5515,7.4730,21.3651,7.5750,13.4975,3.0400,4.143,52.2413,⋯,9.1335,0.1059,4.0880,49.1546,0.5866,7.295,33.1450,7.4290,14.5832,1.4068
RACTLH,0.513,3.9697,0.2723,5.8590,1.5995,5.7180,2.2862,0.1355,0.629,6.1618,⋯,1.6733,0.0049,0.4860,4.4657,0.0377,5.932,2.4926,5.6060,1.9537,0.0491
RACTNY,0.397,0.9533,1.2966,1.5375,0.1327,4.1900,0.7500,0.3303,0.519,2.1010,⋯,0.8985,0.1764,0.4520,0.9665,0.6771,1.479,0.2093,2.7000,0.7339,0.4161
RACTRF,0.500,1.5253,0.9067,4.7270,0.3559,5.8645,1.0508,2.0507,0.954,8.7347,⋯,3.2173,0.7269,0.5280,2.0191,0.5579,4.630,0.5446,5.6930,1.3284,1.7800
RACTRU,1.110,8.9148,1.7533,5.4175,0.7182,6.2145,7.1438,1.9068,1.295,12.2920,⋯,11.4745,0.2106,1.4760,14.1146,2.0287,5.460,1.5131,6.0545,11.0922,1.9874


[1] "/mnt/results/NRG-HN003.sample.values.34.48.csv"
[1] "Finished"


In [219]:
all.measures <- all.measures[,sort(colnames(all.measures))]
head(all.measures)

,all.stroma_all.cell_pdl1.expression,all.stroma_all.pdl1.cell_percent.density,all.stroma_cd8.t.cell_percent.density,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panck.pdl1.cell_percent.density,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_percent.density,all.stroma_treg.cell_percent.density,all.tissue_all.cell_pdl1.expression,all.tissue_all.pdl1.cell_percent.density,⋯,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,inner.tumor.inv.margin_treg.cell_percent.density,outer.tumor.inv.margin_all.cell_pdl1.expression,outer.tumor.inv.margin_all.pdl1.cell_percent.density,outer.tumor.inv.margin_cd8.t.cell_percent.density,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_percent.density,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,outer.tumor.inv.margin_treg.cell_percent.density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.592,2.4819,0.0092,3.0460,0.3137,4.3280,2.1497,0.0046,0.535,1.9324,⋯,0.6375,0.0000,0.6635,2.6947,0.0109,3.166,0.3655,4.2080,2.3074,0.0055
RACTKE,2.937,36.9659,1.5515,7.4730,21.3651,7.5750,13.4975,3.0400,4.143,52.2413,⋯,9.1335,0.1059,4.0880,49.1546,0.5866,7.295,33.1450,7.4290,14.5832,1.4068
RACTLH,0.513,3.9697,0.2723,5.8590,1.5995,5.7180,2.2862,0.1355,0.629,6.1618,⋯,1.6733,0.0049,0.4860,4.4657,0.0377,5.932,2.4926,5.6060,1.9537,0.0491
RACTNY,0.397,0.9533,1.2966,1.5375,0.1327,4.1900,0.7500,0.3303,0.519,2.1010,⋯,0.8985,0.1764,0.4520,0.9665,0.6771,1.479,0.2093,2.7000,0.7339,0.4161
RACTRF,0.500,1.5253,0.9067,4.7270,0.3559,5.8645,1.0508,2.0507,0.954,8.7347,⋯,3.2173,0.7269,0.5280,2.0191,0.5579,4.630,0.5446,5.6930,1.3284,1.7800
RACTRU,1.110,8.9148,1.7533,5.4175,0.7182,6.2145,7.1438,1.9068,1.295,12.2920,⋯,11.4745,0.2106,1.4760,14.1146,2.0287,5.460,1.5131,6.0545,11.0922,1.9874


In [227]:
row.names(all.measures)

[1] "RACTIH" "RACTKE" "RACTLH" "RACTNY" "RACTRF" "RACTRU" "RACTWT" "RACTXK"
 [9] "RACTXV" "RACTYV" "RACUEV" "RACUGF" "RACUVL" "RACUWG" "RACUWR" "RACUWY"
[17] "RACUXG" "RACUZD" "RACUZX" "RACVAN" "RACVCZ" "RACVET" "RACVGZ" "RACVHW"
[25] "RACVHX" "RACVHY" "RACVHZ" "RACVIR" "RACVIS" "RACVIV" "RACVIZ" "RACVJC"
[33] "RACVKA" "RACVLA"

In [ ]:
fn <- file.path(results,sprintf('NRG-HN003.sample.values.%d.%d.csv',
                                dim(all.measures)[1],
                                dim(all.measures)[2]))
print(fn)
write.csv(all.measures,sprintf('./NRG-HN003.sample.values.%d.%d.csv',
                                dim(all.measures)[1],
                                dim(all.measures)[2]))

# Check column names have all comparisons in density and pdl1 expression, 6 tissue x 3 = 18 cell PDL1 expression, 6 tissue * 5 = 30 cell density

In [214]:
temp <- all.measures %>% select(matches('pdl1.expression'))
dim(temp)
print(colnames(temp))

[1] 34 18

 [1] "all.tissue_all.cell_pdl1.expression"                                
 [2] "all.tissue_panck.pdl1.cell_pdl1.expression"                         
 [3] "all.tissue_panckneg.cd3neg.pdl1.cell_pdl1.expression"               
 [4] "all.stroma_all.cell_pdl1.expression"                                
 [5] "all.tumor_all.cell_pdl1.expression"                                 
 [6] "all.stroma_panck.pdl1.cell_pdl1.expression"                         
 [7] "all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression"               
 [8] "all.tumor_panck.pdl1.cell_pdl1.expression"                          
 [9] "all.tumor_panckneg.cd3neg.pdl1.cell_pdl1.expression"                
[10] "all.tumor.invasive.margin_all.cell_pdl1.expression"                 
[11] "all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression"          
[12] "all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression"
[13] "inner.tumor.inv.margin_all.cell_pdl1.expression"                    
[14] "outer.tumor.inv.mar

In [215]:
temp <- all.measures %>% select(matches('.density'))
dim(temp)
print(sort(colnames(temp)))

[1] 34 30

 [1] "all.stroma_all.pdl1.cell_percent.density"                           
 [2] "all.stroma_cd8.t.cell_percent.density"                              
 [3] "all.stroma_panck.pdl1.cell_percent.density"                         
 [4] "all.stroma_panckneg.cd3neg.pdl1.cell_percent.density"               
 [5] "all.stroma_treg.cell_percent.density"                               
 [6] "all.tissue_all.pdl1.cell_percent.density"                           
 [7] "all.tissue_cd8.t.cell_percent.density"                              
 [8] "all.tissue_panck.pdl1.cell_percent.density"                         
 [9] "all.tissue_panckneg.cd3neg.pdl1.cell_percent.density"               
[10] "all.tissue_treg.cell_percent.density"                               
[11] "all.tumor.invasive.margin_all.pdl1.cell_percent.density"            
[12] "all.tumor.invasive.margin_cd8.t.cell_percent.density"               
[13] "all.tumor.invasive.margin_panck.pdl1.cell_percent.density"          
[14] "all.tumor.invasive.

In [217]:
temp <- all.measures %>% select(matches('.density')) %>%
        select(matches('pdl1.cell'))
dim(temp)
print(sort(colnames(temp)))

[1] 34 18

 [1] "all.stroma_all.pdl1.cell_percent.density"                           
 [2] "all.stroma_panck.pdl1.cell_percent.density"                         
 [3] "all.stroma_panckneg.cd3neg.pdl1.cell_percent.density"               
 [4] "all.tissue_all.pdl1.cell_percent.density"                           
 [5] "all.tissue_panck.pdl1.cell_percent.density"                         
 [6] "all.tissue_panckneg.cd3neg.pdl1.cell_percent.density"               
 [7] "all.tumor.invasive.margin_all.pdl1.cell_percent.density"            
 [8] "all.tumor.invasive.margin_panck.pdl1.cell_percent.density"          
 [9] "all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_percent.density"
[10] "all.tumor_all.pdl1.cell_percent.density"                            
[11] "all.tumor_panck.pdl1.cell_percent.density"                          
[12] "all.tumor_panckneg.cd3neg.pdl1.cell_percent.density"                
[13] "inner.tumor.inv.margin_all.pdl1.cell_percent.density"               
[14] "inner.tumor.inv.mar

# Take median of each column

In [12]:
all.measures <- read.csv(file.path(results,'NRG-HN003.sample.values.34.48.csv'),
                        row.names = 1)
head(all.measures)

,all.stroma_all.cell_pdl1.expression,all.stroma_all.pdl1.cell_percent.density,all.stroma_cd8.t.cell_percent.density,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panck.pdl1.cell_percent.density,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_percent.density,all.stroma_treg.cell_percent.density,all.tissue_all.cell_pdl1.expression,all.tissue_all.pdl1.cell_percent.density,⋯,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,inner.tumor.inv.margin_treg.cell_percent.density,outer.tumor.inv.margin_all.cell_pdl1.expression,outer.tumor.inv.margin_all.pdl1.cell_percent.density,outer.tumor.inv.margin_cd8.t.cell_percent.density,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_percent.density,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,outer.tumor.inv.margin_treg.cell_percent.density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.592,2.4819,0.0092,3.0460,0.3137,4.3280,2.1497,0.0046,0.535,1.9324,⋯,0.6375,0.0000,0.6635,2.6947,0.0109,3.166,0.3655,4.2080,2.3074,0.0055
RACTKE,2.937,36.9659,1.5515,7.4730,21.3651,7.5750,13.4975,3.0400,4.143,52.2413,⋯,9.1335,0.1059,4.0880,49.1546,0.5866,7.295,33.1450,7.4290,14.5832,1.4068
RACTLH,0.513,3.9697,0.2723,5.8590,1.5995,5.7180,2.2862,0.1355,0.629,6.1618,⋯,1.6733,0.0049,0.4860,4.4657,0.0377,5.932,2.4926,5.6060,1.9537,0.0491
RACTNY,0.397,0.9533,1.2966,1.5375,0.1327,4.1900,0.7500,0.3303,0.519,2.1010,⋯,0.8985,0.1764,0.4520,0.9665,0.6771,1.479,0.2093,2.7000,0.7339,0.4161
RACTRF,0.500,1.5253,0.9067,4.7270,0.3559,5.8645,1.0508,2.0507,0.954,8.7347,⋯,3.2173,0.7269,0.5280,2.0191,0.5579,4.630,0.5446,5.6930,1.3284,1.7800
RACTRU,1.110,8.9148,1.7533,5.4175,0.7182,6.2145,7.1438,1.9068,1.295,12.2920,⋯,11.4745,0.2106,1.4760,14.1146,2.0287,5.460,1.5131,6.0545,11.0922,1.9874


In [13]:
mv <- apply(all.measures, 2, median)
mv

all.stroma_all.cell_pdl1.expression 
                                                            1.65250 
                           all.stroma_all.pdl1.cell_percent.density 
                                                           16.09175 
                              all.stroma_cd8.t.cell_percent.density 
                                                            2.61545 
                         all.stroma_panck.pdl1.cell_pdl1.expression 
                                                            6.02150 
                         all.stroma_panck.pdl1.cell_percent.density 
                                                            2.15030 
               all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                            6.30150 
               all.stroma_panckneg.cd3neg.pdl1.cell_percent.density 
                                                           10.63280 
                               all.stroma_treg.cell_percent.density 
                                                            3.26595 
                                all.tissue_all.cell_pdl1.expression 
                                                            1.80250 
                           all.tissue_all.pdl1.cell_percent.density 
                                                           21.47225 
                              all.tissue_cd8.t.cell_percent.density 
                                                            1.83605 
                         all.tissue_panck.pdl1.cell_pdl1.expression 
                                                            5.80650 
                         all.tissue_panck.pdl1.cell_percent.density 
                                                            7.53825 
               all.tissue_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                            6.24150 
               all.tissue_panckneg.cd3neg.pdl1.cell_percent.density 
                                                           11.53480 
                               all.tissue_treg.cell_percent.density 
                                                            2.16550 
                 all.tumor.invasive.margin_all.cell_pdl1.expression 
                                                            2.14700 
            all.tumor.invasive.margin_all.pdl1.cell_percent.density 
                                                           28.17730 
               all.tumor.invasive.margin_cd8.t.cell_percent.density 
                                                            1.93390 
          all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression 
                                                            5.80725 
          all.tumor.invasive.margin_panck.pdl1.cell_percent.density 
                                                           12.38335 
all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                            6.26800 
all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_percent.density 
                                                           12.36025 
                all.tumor.invasive.margin_treg.cell_percent.density 
                                                            2.12605 
                                 all.tumor_all.cell_pdl1.expression 
                                                            2.28850 
                            all.tumor_all.pdl1.cell_percent.density 
                                                           28.76540 
                               all.tumor_cd8.t.cell_percent.density 
                                                            0.96075 
                          all.tumor_panck.pdl1.cell_pdl1.expression 
                                                            5.65050 
                          all.tumor_panck.pdl1.cell_percent.density 
                                                           17.48560 
                all.tumor_panc

In [20]:
high.low <- all.measures %>% 
  reframe(across(everything(), ~ ifelse(. > median(., na.rm = TRUE), "HIGH", "LOW"))) 
row.names(high.low) <- row.names(all.measures)
head(high.low,n=10)


,all.stroma_all.cell_pdl1.expression,all.stroma_all.pdl1.cell_percent.density,all.stroma_cd8.t.cell_percent.density,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panck.pdl1.cell_percent.density,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_percent.density,all.stroma_treg.cell_percent.density,all.tissue_all.cell_pdl1.expression,all.tissue_all.pdl1.cell_percent.density,⋯,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,inner.tumor.inv.margin_treg.cell_percent.density,outer.tumor.inv.margin_all.cell_pdl1.expression,outer.tumor.inv.margin_all.pdl1.cell_percent.density,outer.tumor.inv.margin_cd8.t.cell_percent.density,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_percent.density,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,outer.tumor.inv.margin_treg.cell_percent.density
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
RACTIH,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,⋯,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW
RACTKE,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,⋯,HIGH,LOW,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,HIGH,LOW
RACTLH,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,⋯,LOW,LOW,LOW,LOW,LOW,HIGH,LOW,LOW,LOW,LOW
RACTNY,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,⋯,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW
RACTRF,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,⋯,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW
RACTRU,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,⋯,HIGH,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW,LOW
RACTWT,LOW,LOW,LOW,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,⋯,HIGH,HIGH,LOW,LOW,LOW,HIGH,HIGH,HIGH,LOW,HIGH
RACTXK,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,HIGH,HIGH,HIGH,⋯,LOW,LOW,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,HIGH
RACTXV,HIGH,HIGH,LOW,LOW,HIGH,HIGH,HIGH,LOW,LOW,LOW,⋯,LOW,LOW,HIGH,LOW,LOW,LOW,LOW,HIGH,HIGH,LOW


,RACTIH,RACTKE,RACTLH,RACTNY,RACTRF,RACTRU,RACTWT,RACTXK,RACTXV,RACTYV,⋯,RACVHX,RACVHY,RACVHZ,RACVIR,RACVIS,RACVIV,RACVIZ,RACVJC,RACVKA,RACVLA
all.stroma_all.cell_pdl1.expression,LOW,HIGH,LOW,LOW,LOW,LOW,LOW,HIGH,HIGH,HIGH,⋯,HIGH,LOW,HIGH,HIGH,LOW,HIGH,LOW,LOW,HIGH,LOW
all.stroma_all.pdl1.cell_percent.density,LOW,HIGH,LOW,LOW,LOW,LOW,LOW,HIGH,HIGH,HIGH,⋯,HIGH,LOW,HIGH,HIGH,LOW,HIGH,LOW,LOW,HIGH,LOW
all.stroma_cd8.t.cell_percent.density,LOW,LOW,LOW,LOW,LOW,LOW,LOW,HIGH,LOW,HIGH,⋯,LOW,LOW,HIGH,LOW,LOW,HIGH,HIGH,LOW,HIGH,LOW
all.stroma_panck.pdl1.cell_pdl1.expression,LOW,HIGH,LOW,LOW,LOW,LOW,HIGH,LOW,LOW,HIGH,⋯,LOW,HIGH,HIGH,HIGH,LOW,HIGH,LOW,LOW,HIGH,LOW
all.stroma_panck.pdl1.cell_percent.density,LOW,HIGH,LOW,LOW,LOW,LOW,HIGH,HIGH,HIGH,HIGH,⋯,HIGH,LOW,HIGH,HIGH,LOW,HIGH,HIGH,LOW,LOW,LOW
all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,LOW,HIGH,LOW,LOW,LOW,LOW,HIGH,HIGH,HIGH,HIGH,⋯,LOW,HIGH,LOW,HIGH,LOW,HIGH,LOW,LOW,LOW,LOW


In [23]:
colSums(high.low == 'LOW')

all.stroma_all.cell_pdl1.expression 
                                                                 17 
                           all.stroma_all.pdl1.cell_percent.density 
                                                                 17 
                              all.stroma_cd8.t.cell_percent.density 
                                                                 17 
                         all.stroma_panck.pdl1.cell_pdl1.expression 
                                                                 17 
                         all.stroma_panck.pdl1.cell_percent.density 
                                                                 17 
               all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                                 17 
               all.stroma_panckneg.cd3neg.pdl1.cell_percent.density 
                                                                 17 
                               all.stroma_treg.cell_percent.density 
                                                                 17 
                                all.tissue_all.cell_pdl1.expression 
                                                                 17 
                           all.tissue_all.pdl1.cell_percent.density 
                                                                 17 
                              all.tissue_cd8.t.cell_percent.density 
                                                                 17 
                         all.tissue_panck.pdl1.cell_pdl1.expression 
                                                                 17 
                         all.tissue_panck.pdl1.cell_percent.density 
                                                                 17 
               all.tissue_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                                 17 
               all.tissue_panckneg.cd3neg.pdl1.cell_percent.density 
                                                                 17 
                               all.tissue_treg.cell_percent.density 
                                                                 17 
                 all.tumor.invasive.margin_all.cell_pdl1.expression 
                                                                 17 
            all.tumor.invasive.margin_all.pdl1.cell_percent.density 
                                                                 17 
               all.tumor.invasive.margin_cd8.t.cell_percent.density 
                                                                 17 
          all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression 
                                                                 17 
          all.tumor.invasive.margin_panck.pdl1.cell_percent.density 
                                                                 17 
all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression 
                                                                 17 
all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_percent.density 
                                                                 17 
                all.tumor.invasive.margin_treg.cell_percent.density 
                                                                 17 
                                 all.tumor_all.cell_pdl1.expression 
                                                                 17 
                            all.tumor_all.pdl1.cell_percent.density 
                                                                 17 
                               all.tumor_cd8.t.cell_percent.density 
                                                                 17 
                          all.tumor_panck.pdl1.cell_pdl1.expression 
                                                                 17 
                          all.tumor_panck.pdl1.cell_percent.density 
                                                                 17 
                all.tumor_panc

# Write raw and median high/low in transponsed matrix

In [32]:
all.meas.t <- t(all.measures)
fn <- file.path(results,sprintf('NRG-HN003.raw.sample.values.t.%d.%d.csv',
                                dim(all.meas.t)[1],
                                dim(all.meas.t)[2]))
print(fn)
write.csv(all.meas.t, fn)

high.low.t <- t(high.low)
fn<- file.path(results,sprintf('NRG-HN003.high.low.values.t.%d.%d.csv',
                                dim(high.low.t)[1],
                                dim(high.low.t)[2]))
print(fn)
write.csv(high.low.t,fn)

[1] "/mnt/results/NRG-HN003.raw.sample.values.t.48.34.csv"
[1] "/mnt/results/NRG-HN003.high.low.values.t.48.34.csv"


# Examine patients LOW for everything

In [35]:
all.low <- row.names(high.low[rowSums(high.low == "HIGH") == 0,])
all.measures[all.low,]

,all.stroma_all.cell_pdl1.expression,all.stroma_all.pdl1.cell_percent.density,all.stroma_cd8.t.cell_percent.density,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panck.pdl1.cell_percent.density,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_percent.density,all.stroma_treg.cell_percent.density,all.tissue_all.cell_pdl1.expression,all.tissue_all.pdl1.cell_percent.density,⋯,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,inner.tumor.inv.margin_treg.cell_percent.density,outer.tumor.inv.margin_all.cell_pdl1.expression,outer.tumor.inv.margin_all.pdl1.cell_percent.density,outer.tumor.inv.margin_cd8.t.cell_percent.density,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_percent.density,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_percent.density,outer.tumor.inv.margin_treg.cell_percent.density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.592,2.4819,0.0092,3.0460,0.3137,4.3280,2.1497,0.0046,0.535,1.9324,⋯,0.6375,0.0000,0.6635,2.6947,0.0109,3.1660,0.3655,4.2080,2.3074,0.0055
RACTNY,0.397,0.9533,1.2966,1.5375,0.1327,4.1900,0.7500,0.3303,0.519,2.1010,⋯,0.8985,0.1764,0.4520,0.9665,0.6771,1.4790,0.2093,2.7000,0.7339,0.4161
RACVAN,1.057,6.9331,2.0071,5.5125,1.4183,6.0955,4.6808,2.1962,1.223,8.1039,⋯,1.4140,0.2020,1.1040,8.1773,0.7189,5.4175,2.6648,5.6725,5.0871,1.3816
RACVCZ,0.615,3.2382,1.5378,5.0230,0.4954,6.0000,2.3692,0.9473,0.692,3.6918,⋯,1.6100,0.1146,0.7600,4.5161,1.3815,5.0150,0.9755,5.7080,3.1690,0.7314
RACVLA,0.662,7.3580,0.0972,3.3760,0.0833,4.6465,7.2470,0.0000,0.544,5.9842,⋯,0.3086,0.0000,0.5890,7.7134,0.0000,3.1410,0.1714,4.5660,7.5077,0.0000


In [54]:
sample.n <- df %>% 
    group_by(Sample.Name,fn,all.tumor.stroma) %>%
    summarize(n = n()) %>%
    group_by(Sample.Name) %>%
    summarize(n.rois = n(),total.cells = sum(n)) %>% 
    tibble::column_to_rownames(var='Sample.Name')
# sample.n

ts<- df %>%  group_by(Sample.Name,tissue.compartment) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "n.tumor.cells",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "n.stroma.cells",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    summarize(n=sum(n)) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma"),
               values_from = "n") %>%
    tibble::column_to_rownames(var='Sample.Name')
sample.n <- bind_cols(sample.n, ts)
t<-c()
for (nm in row.names(sample.n)){
    t <- c(t,str_split_fixed(nm,'_',n=2)[1])
}
row.names(sample.n) <- t

fn <- file.path(results,"NRG-HN003-sample-summary.csv")
print(fn)
write.csv(sample.n,fn)
head(sample.n)

`summarise()` has grouped output by 'Sample.Name', 'fn'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


[1] "/mnt/results/NRG-HN003-sample-summary.csv"


,n.rois,total.cells,n.stroma.cells,n.tumor.cells
,<int>,<int>,<int>,<int>
RACTIH,24,55579,21677,33902
RACTKE,570,1098425,568216,530209
RACTLH,118,281348,153487,127861
RACTNY,54,116565,69335,47230
RACTRF,20,27259,11801,15458
RACTRU,76,168174,130323,37851


In [50]:
sample.n[all.low,]

,n.rois,total.cells,n.stroma.cells,n.tumor.cells
,<int>,<int>,<int>,<int>
RACTIH,24,55579,21677,33902
RACTNY,54,116565,69335,47230
RACVAN,140,276695,133822,142873
RACVCZ,188,543253,332433,210820
RACVLA,4,9007,7203,1804


In [40]:
colnames(df)

[1] "X"                  "Sample.Name"        "fn"                
 [4] "Cell.ID"            "cell.type"          "all.tumor.stroma"  
 [7] "tissue.compartment" "pdl1.expression"    "pd1.expression"    
[10] "combined.pheno"     "is.pdl1.cell"       "is.panck.cell"     
[13] "pdl1.cell"          "panck.pdl1.cell"

In [142]:
colnames(sub)

[1] "X"                  "Sample.Name"        "fn"                
 [4] "Cell.ID"            "cell.type"          "all.tumor.stroma"  
 [7] "tissue.compartment" "pdl1.expression"    "pd1.expression"    
[10] "combined.pheno"     "is.pdl1.cell"       "is.panck.cell"     
[13] "pdl1.cell"          "panck.pdl1.cell"

In [208]:
sub %>%  group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,panck.pdl1.cell,panckneg.cd3neg.pdl1.cell
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.7107,1.2127
RACTKE_Scan1.qptiff,38.1146,12.9813


In [206]:
sub %>%  group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,panck.pdl1.cell,panckneg.cd3neg.pdl1.cell
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.7107,1.2127
RACTKE_Scan1.qptiff,38.1146,12.9813


In [172]:
sub %>%  group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    select(-matches("other")) %>%
    rename_with(~paste0(.,'_pdl1.expression'),everything())

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,panck.pdl1.cell_pdl1.expression,panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,3.278,4.041
RACTKE_Scan1.qptiff,6.648,6.917


In [148]:
sub  %>%       mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin,panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0)) %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

`summarise()` has grouped output by 'Sample.Name',
'tissue.all.invasive.margin'. You can override using the `.groups` argument.


,all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression,all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,3.186,3.999
RACTKE_Scan1.qptiff,6.535,6.811


In [146]:
# Create a sample data.frame
df.test <- data.frame(x = 1:5)

# Rename the rows
df.test %>% 
  mutate(y = case_when(
    x == 1 ~ "one",
    x == 2 ~ "two",
    x == 3 ~ "three",
    TRUE ~ "other"
  ))

x,y
<int>,<chr>
1,one
2,two
3,three
4,other
5,other
